**Sustainable Software Development, block course, March 2021**  
*Scientific Software Center, Institute for Scientific Computing, Dr. Inga Ulusoy*

# Analysis of the data

Imagine you perform a "measurement" of some type and obtain "scientific data". You know what your data represents, but you have only a vague idea how different features in the data are connected, and what information you can extract from the data.

You would start first with going through the data, making sure your data set is complete and that the result is reasonable. Imagine this already happened.

In the next step, you would inspect your data more closely and try to identify structures. That is the step that we are focusing on in this unit.

In the `data` folder, you will find several data files (`*.t` and `*.dat`). These are data files generated through some "new approach" that hasn't been used in your lab before. No previous analysis software exists, and you are going to establish a protocol for this "new approach" and "publish your results".

The data can be grouped into two categories: 
1. data to be analyzed using statistical methods;
2. data to be analyzed using numerical methods.

In your hypothetical lab, you are an "expert" in one particular "method", and your co-worker is an "expert" in the other. Combined these two methods will lead to much more impactful results than if only one of you analyzed the data. Now, the task in this course is to be solved collaboratively with your team member working on one of the analysis approaches, and you working on the other. You will both implement functionality into the same piece of "software", but do so collaboratively through git.

As you do not know yet which analysis is most meaningful for your data, and how to implement it, you will start with a jupyter notebook. You and your team member will work on the same notebook that will be part of a github repository for your project. This is the task for today. Discuss with your team members who will work on the statistical and who on the numerical analysis.

## Step 1

Accept the assignment through GitHub classroom.

## Step 2

Clone the repository to your local machine.

## Step 3

Start working on task 1 for your analysis approach. 

## Step 4

Create your own branch of the repository and commit your changes to your branch; push to the remote repository.

## Step 5

Open a `pull request` so your team member can review your implementation. Likewise, your team member will ask you to review theirs.

## Step 6

Merge the changes in your branch into `main`. Resolve conflicts.

## Step 7

Repeat working on task; committing and pushing to your previously generated branch or a new branch; open a pull request; merge with main; until you have finished all the tasks in your analysis approach. Delete obsolete branches.

# Start of the analysis notebook

**Author : Julian Heidecke**  
*Date : 23.06.2025*  
*Affiliation : IWR Heidelberg University*  

Place the required modules in the top, followed by required constants and global functions.

In [ ]:
# required modules
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
# constants and global functions
THRESHOLD = 1e-4

In [ ]:
# reading of the data files
expec = pd.read_csv('../data/expec.t', sep=r'\s+')
npop = pd.read_csv('../data/npop.t', sep=r'\s+')
table = pd.read_csv('../data/table.dat', sep=r'\s+')

# Statistical analysis

Find correlations in the data sets. Analyse the data statistically and plot your results.  

Here we would want to do everything with pandas and leave the data in a dataframe. The files that are relevant to you are `expect.t`, `npop.t` and `table.dat`.

### Task 1: Read in expec.t and plot relevant data

In [ ]:
# Display the first few rows to understand the structure
print(expec.head())
print(expec.dtypes)
# Plotting
# Set 'time' as the index for cleaner plots
expec.set_index('time', inplace=True)
# Plot all numeric columns
expec.plot(figsize=(10, 6), marker='o')
plt.title('Plot from expec.t')
plt.xlabel('Time')
plt.ylabel('Values')
plt.grid(True)
plt.tight_layout()
plt.show()

We can discard the entries norm, \<x>, and \<y> as these are mostly constant.

In [ ]:
# eliminate columns based on the variance: if the variance of the values
# in a column is below a given threshold, that column is discarded
variances = expec.var()
expec_filtered = expec.loc[:, variances > THRESHOLD]

print("Dropped columns:", variances[variances <= THRESHOLD].index.tolist())
print("Remaining columns:", expec_filtered.columns.tolist())

### Task 2: Create plots of the relevant data and save as .pdf.

In [ ]:
# create plots
expec_filtered.plot(figsize=(10, 6), marker='o')
plt.title('Plot of High-Variance Columns from expec.t')
plt.xlabel('Time')
plt.ylabel('Values')
plt.grid(True)
plt.tight_layout()

plt.savefig('filtered_plot.pdf')  # Save before show
plt.show()

### Task 3: Read in file `npop.t` and analyze correlations in the data

In [ ]:
# Check structure
print(npop.head())
print(npop.dtypes)

In [ ]:
# discard all columns with variance below a set threshold - we can consider them as constant
variances = npop.var()
npop_filtered = npop.loc[:, variances > THRESHOLD]
npop_filtered.head()

Plot the remaining columns. Seaborn prefers "long format" (one column for all measurement values, one column to indicate the type) as input, whereas the cvs is in "wide format" (one column per measurement type).

In [ ]:
npop_long = npop_filtered.melt(id_vars='time', var_name='Molecule', value_name='Population')
npop_long.head()

In [ ]:
# Step 3: Plot
plt.figure(figsize=(12, 6))
sns.lineplot(data=npop_long, x='time', y='Population', hue='Molecule', marker='o')

plt.title('Molecular Population Over Time')
plt.xlabel('Time')
plt.ylabel('Population')
plt.grid(True)
plt.tight_layout()
plt.show()

## Quantify the pairwise correlation in the data

- negative correlation: y values decrease for increasing x - large values of one feature correspond to small values of the other feature
- weak or no correlation: no trend observable, association between two features is hardly observable
- positive correlation: y values increase for decreasing x - small values of one feature correspond to small values of the other feature

Remember that correlation does not indicate causation - the reason that two features are associated can lie in their dependence on same factors.

Correlate the value pairs using Pearson's $r$. Pearson's $r$ is a measure of the linear relationship between features:

$r = \frac{\sum_i(x_i − \bar{x})(y_i − \bar{y})}{\sqrt{\sum_i(x_i − \bar{x})^2 \sum_i(y_i − \bar{y})^2}}$

Here, $\bar{x}$ and $\bar{y}$ indicate mean values. $i$ runs over the whole data set. For a positive correlation, $r$ is positive, and negative for a negative correlation, with minimum and maximum values of -1 and 1, indicating a perfectly linear relationship. Weakly or not correlated features are characterized by $r$-values close to 0.

Other measures of correlation that can be used are Spearman's rank (value pairs follow monotonic function) or Kendall's $\tau$ (measures ordinal association), but they do not apply here. You can also define measures yourself.

In [ ]:
# print the correlation matrix
correlation_matrix = npop_filtered.corr(method='pearson')
print(correlation_matrix)

The diagonal values tell us that each value is perfectly correlated with itself. We are not interested in the diagonal values and also not in the correlation with time. We also need to get rid of redundant entries. Finally, we need to find the value pairs that exhibit the highest linear correlation. We still want to know if it is positive or negative correlation, so we cannot get rid of the sign.

In [ ]:
# get rid of time column, lower triangular and diagonal entries of the correlation matrix
# sort the remaing values according to their absolute value, but keep the sign
correlation_matrix = correlation_matrix.drop(columns=['time'], index=['time'])
correlation_matrix = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))
correlation_matrix = correlation_matrix.stack().reset_index()
correlation_matrix.head()
correlation_matrix.columns = ['MO1', 'MO2', 'correlation']
correlation_matrix['abs_correlation'] = correlation_matrix['correlation'].abs()
correlation_matrix = correlation_matrix.sort_values(by='abs_correlation', ascending=False)
print("Sorted correlation matrix:")
print(correlation_matrix)

Note that the entries in the left column are not repeated if they do not change from the row above (so the fourth feature pair is MO3 and MO6).

### Task 4: Print the resulting data to a file

In [ ]:
# write to file
correlation_matrix.to_csv('correlation_matrix.csv', index=False)

### Task 5: Calculate the Euclidean distance (L2 norm) for the vectors in `table.dat`


The Euclidean distance measures the distance between to objects that are not points:

$d(p,q) = \sqrt{\left(p-q\right)^2}$

In this case, consider each of the columns in table.dat as a vector in Euclidean space, where column $r(x)$ and column $v(x)$ denote a pair of vectors that should be compared, as well as $r(y)$ and $v(y)$, and r(z) and v(z).

(Background: These are dipole moment components in different gauges, the length and velocity gauge.)

In [ ]:
# read in table.dat - I suggest reading it as a numpy array
# replace the NaNs by zero
table = table.fillna(0)
table.head()

Now calculate how different the vectors in column 2 are from column 3, column 4 from column 5, and column 6 from column 7.

In [ ]:
# calculate the Euclidean distance
dist_2_3 = np.linalg.norm(table.iloc[:, 1] - table.iloc[:, 2])
dist_4_5 = np.linalg.norm(table.iloc[:, 3] - table.iloc[:, 4])
dist_6_7 = np.linalg.norm(table.iloc[:, 5] - table.iloc[:, 6])

print(f"Euclidean distance between column 2 and 3: {dist_2_3}")
print(f"Euclidean distance between column 4 and 5: {dist_4_5}")
print(f"Euclidean distance between column 6 and 7: {dist_6_7}")

In [ ]:
# plot the result and save to a .pdf
distances = [dist_2_3, dist_4_5, dist_6_7]
labels = ['Col 2 vs 3', 'Col 4 vs 5', 'Col 6 vs 7']

# Create bar plot
plt.figure(figsize=(6, 4))
plt.bar(labels, distances, color='skyblue')
plt.ylabel('Euclidean Distance')
plt.title('Euclidean Distances Between Column Pairs')

# Save plot as PDF
plt.savefig('euclidean_distances.pdf')
plt.show()

In [ ]:
# print the result to a file
with open('euclidean_distances.txt', 'w') as f:
    f.write(f"Euclidean distance between column 2 and 3: {dist_2_3}\n")
    f.write(f"Euclidean distance between column 4 and 5: {dist_4_5}\n")
    f.write(f"Euclidean distance between column 6 and 7: {dist_6_7}\n")

# Numerical analysis

Analyze the data using autocorrelation functions and discrete Fourier transforms. Plot your results.

In [ ]:
# define some global functions

### Task 1: Read in `efield.t` and Fourier-transform relevant columns

In [ ]:
# read and plot efield.t
data = np.genfromtxt('../data/efield.t', delimiter=None, skip_header=1)
print(data)

Here we are interested in column 2 since the others are constant.

In [ ]:
# discard the columns with variance below threshold - these are considered constant
variances = np.var(data, axis=0)
filtered_data = data[:, variances > THRESHOLD]
print(filtered_data)

In [ ]:
# discrete Fourier transform of the remaining column: You only need the real frequencies
time = data[:, 0]
signals = filtered_data[:, 1]

### Task 2: Generate a plot of your results to be saved as pdf.

In [ ]:
# plot your results

### Task 3: Calculate the autocorrelation function from nstate_i.t
The autocorrelation function measures how correlated subsequent vectors are with an initial vector; ie. 

$\Psi_{corr} = \langle \Psi(t=0) | \Psi(t) \rangle = \int_0^{tfin} \Psi(0)^* \Psi(t) dt$

Since we are in a numerical representation, the integral can be replaced with a sum; and the given vectors are already normalized.

In [ ]:
# read in as numpy array

In [ ]:
# store the time column (column 0) in a vector and drop from array

In [ ]:
# correct the data representation: this is in fact a complex matrix
# the real part of each matrix column is contained in numpy array column 0, 2, 4, 6, ...
# the imaginary part of each matrix column is contained in numpy array column 1, 3, 5, 7, ...
# convert the array that was read as dtype=float into a dtype=complex array

In [ ]:
# for the autocorrelation function, we want the overlap between the first vector at time 0 and all
# subsequent vectors at later times - the sum of the product of initial and subsequent vectors for each time step

### Task 4: Generate a plot of your results to be saved as pdf.

In [ ]:
# plot the autocorrelation function - real, imaginary and absolute part

### Task 5: Discrete Fourier transform of the autocorrelation function

In [ ]:
# discrete Fourier-transform the autocorrelation function - now we need all frequency components,
# also the negative ones

### Task 6: Generate a plot of your results to be saved as pdf.

In [ ]:
# plot the power spectrum (abs**2)